In [39]:
!pip install lxml
!pip install pytesseract Pillow
!pip install atlassian-python-api
!pip install langchain
!pip install tokenizers
!pip install redis
!pip install python-dotenv

In [40]:
from langchain.document_loaders import ConfluenceLoader
import lxml
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

loader = ConfluenceLoader(
    url="https://confluence.zhenguanyu.com/",
    token=os.environ['CONFLUENCE_TOKEN'],
    cloud=False
)
page_id = 192873905
raw_cql = f"type = page AND ancestor = {page_id}"
documents = loader.load(space_key="CON", include_attachments=False, limit=50)

start


TypeError: string indices must be integers, not 'str'

In [3]:
print(len(documents))

424


In [4]:
from langchain.vectorstores.redis import Redis
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, length_function = len)
docs = text_splitter.split_documents(documents)
print(len(docs))

535


In [6]:
from langchain.embeddings.openai import OpenAIEmbeddings
import os

embeddings = OpenAIEmbeddings(deployment="embeddings_model", chunk_size=1)
db = Redis.from_documents(docs, embeddings, redis_url="redis://localhost:6379", index_name='chat_confluence_new')

In [29]:
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain, _get_chat_history
from typing import Dict, Any, Optional, List
from langchain.callbacks.manager import CallbackManagerForChainRun
from langchain.schema import Document

class CustomConversationalRetrievalChain(ConversationalRetrievalChain):
    last_source_documents: List[Document] = None

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.last_source_documents = None
        
    def _call(self, inputs: Dict[str, Any], run_manager: Optional[CallbackManagerForChainRun]=None) -> Dict[str, Any]:
        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        question = inputs['question']
        get_chat_history = self.get_chat_history or _get_chat_history
        chat_history_str = get_chat_history(inputs['chat_history'])
        if chat_history_str:
            callbacks = _run_manager.get_child()
            new_question = self.question_generator.run(question=question, chat_history=chat_history_str, callbacks=callbacks)
        else:
            new_question = question
        docs = self._get_docs(new_question, inputs)
        new_inputs = inputs.copy()
        new_inputs['question'] = new_question
        new_inputs['chat_history'] = chat_history_str
        answer = self.combine_docs_chain.run(input_documents=docs, callbacks=_run_manager.get_child(), **new_inputs)
        self.last_source_documents = docs
        return {self.output_key: answer}



In [38]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.vectorstores.redis import Redis
from langchain.embeddings.openai import OpenAIEmbeddings
import langchain
import os
embeddings = OpenAIEmbeddings(deployment="embeddings_model", chunk_size=1)

query_db = Redis.from_existing_index(embeddings, redis_url="redis://localhost:6379", index_name='chat_confluence_new')
retriever = query_db.as_retriever()
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv, find_dotenv

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
llm = AzureChatOpenAI(deployment_name="chat_confluence",model_name="gpt-35-turbo", verbose=True)
qa = CustomConversationalRetrievalChain.from_llm(llm, retriever, memory=memory)
langchain.debug=True
res1 = qa({"question": "How to create a service?"})
# print(f"answer is {res1}, source_docs is {qa.last_source_documents}")
res2 = qa({"question": ""})
# print(f"res2 is {res2}, source_docs are {qa.last_source_documents}")
langchain.debug=False

[chain/start] [1:RunTypeEnum.chain:CustomConversationalRetrievalChain] Entering Chain run with input:
{
  "question": "How to create a service?",
  "chat_history": []
}
[chain/start] [1:RunTypeEnum.chain:CustomConversationalRetrievalChain > 2:RunTypeEnum.chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:RunTypeEnum.chain:CustomConversationalRetrievalChain > 2:RunTypeEnum.chain:StuffDocumentsChain > 3:RunTypeEnum.chain:LLMChain] Entering Chain run with input:
{
  "question": "How to create a service?",
  "context": "1 \u5982\u4f55\u5728 Console \u4e0a\u65b0\u5efa\u670d\u52a1\uff0c\u9700\u8981\u63d0\u4f9b\u4ec0\u4e48\u4fe1\u606f\u70b9\u51fbConsole\u53f3\u4e0a\u89d2\u7684oncall\uff0c\u63d0oncall\u5904\u7406\u3002oncall\u9700\u8981\u586b\u5199\u7684\u4fe1\u606f\u5982\u4e0b\uff1a\u7cfb\u7edf\uff1aConsole\u7c7b\u578b\uff1a\u95ee\u9898\u53cd\u9988\u7c7b\u6982\u8981\uff1a\u65b0\u5efa\u670d\u52a1gerrit\u9879\u76ee\u540d\uff1a\u4f60\u7684\u9879\u76ee\u540d\uff0c

[llm/end] [1:RunTypeEnum.chain:CustomConversationalRetrievalChain > 2:RunTypeEnum.chain:StuffDocumentsChain > 3:RunTypeEnum.chain:LLMChain > 4:RunTypeEnum.llm:AzureChatOpenAI] [4.40s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To create a service, click on the \"New Service\" button on the service list page. In the pop-up window, you will need to provide the following information: system, console type, summary, gerrit project name, service name, description, service type (HTTP/RPC, static service, or XXL-Job service), and service port. After submitting the request, a specialist will handle it within 10 minutes. Once the service is created, you can apply for machine resources by clicking on \"Apply for Service Platform Service Order\" for service platform deployment or \"Modify K8s Service Environment Information\" for K8s deployment. If you choose the latter, you can simply fill in the corresponding information and save it without needing any furth

[chain/start] [1:RunTypeEnum.chain:CustomConversationalRetrievalChain > 4:RunTypeEnum.chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:RunTypeEnum.chain:CustomConversationalRetrievalChain > 4:RunTypeEnum.chain:StuffDocumentsChain > 5:RunTypeEnum.chain:LLMChain] Entering Chain run with input:
{
  "question": "What information do I need to provide when creating a service?",
  "context": "1 \u5982\u4f55\u5728 Console \u4e0a\u65b0\u5efa\u670d\u52a1\uff0c\u9700\u8981\u63d0\u4f9b\u4ec0\u4e48\u4fe1\u606f\u70b9\u51fbConsole\u53f3\u4e0a\u89d2\u7684oncall\uff0c\u63d0oncall\u5904\u7406\u3002oncall\u9700\u8981\u586b\u5199\u7684\u4fe1\u606f\u5982\u4e0b\uff1a\u7cfb\u7edf\uff1aConsole\u7c7b\u578b\uff1a\u95ee\u9898\u53cd\u9988\u7c7b\u6982\u8981\uff1a\u65b0\u5efa\u670d\u52a1gerrit\u9879\u76ee\u540d\uff1a\u4f60\u7684\u9879\u76ee\u540d\uff0c\u5982infra-fake-project\u670d\u52a1\u540d\uff1a\u4f60\u7684\u670d\u52a1\u540d\uff0c\u5982\uff1ainfra-fake-service\u63cf\u8ff0\uff1

[llm/end] [1:RunTypeEnum.chain:CustomConversationalRetrievalChain > 4:RunTypeEnum.chain:StuffDocumentsChain > 5:RunTypeEnum.chain:LLMChain > 6:RunTypeEnum.llm:AzureChatOpenAI] [2.88s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To create a service on Console, you need to click oncall on the upper right corner of the Console and fill in the following information in the oncall form:\n- System\n- Console type\n- Summary\n- Gerrit project name\n- Service name\n- Description\n- Service type (http/rpc/static service)\n- Service port\n\nAfter submitting the oncall form, a specialist will handle the request within 10 minutes.",
        "generation_info": null,
        "message": {
          "content": "To create a service on Console, you need to click oncall on the upper right corner of the Console and fill in the following information in the oncall form:\n- System\n- Console type\n- Summary\n- Gerrit project name\n- Service name\n- Description\n- Service t